# Day 14
## Part 1
Don't bother creating the string, just create a running counter of characters and depth first search.

In [4]:
from collections import Counter

def parse_data(s):
    lines = s.strip().splitlines()
    template = lines[0].strip()
    insertions = {}
    for line in lines[2:]:
        xs, y = line.strip().split(' -> ')
        insertions[tuple(xs)] = y
    return template, insertions

test_string = '''
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
'''

test_data = parse_data(test_string)
test_data

('NNCB',
 {('C', 'H'): 'B',
  ('H', 'H'): 'N',
  ('C', 'B'): 'H',
  ('N', 'H'): 'C',
  ('H', 'B'): 'C',
  ('H', 'C'): 'B',
  ('H', 'N'): 'C',
  ('N', 'N'): 'C',
  ('B', 'H'): 'H',
  ('N', 'C'): 'B',
  ('N', 'B'): 'B',
  ('B', 'N'): 'B',
  ('B', 'B'): 'N',
  ('B', 'C'): 'B',
  ('C', 'C'): 'N',
  ('C', 'N'): 'C'})

In [10]:
def count_elements(steps, template, insertions):
    counts = Counter(template)
    stack = [(x, y, steps) for x, y in zip(template, template[1:])]
    while stack:
        x, y, n = stack.pop()
        insert = insertions[(x, y)]
        counts[insert] += 1
        if n > 1:
            stack.append((x, insert, n - 1))
            stack.append((insert, y, n - 1))
    return counts

In [11]:
count_elements(10, *test_data)

Counter({'N': 865, 'C': 298, 'B': 1749, 'H': 161})

In [14]:
def part_1(template, insertions):
    counts = count_elements(10, template, insertions)
    return max(counts.values()) - min(counts.values())

assert part_1(*test_data) == 1588

In [15]:
data = parse_data(open('input', 'r').read())
part_1(*data)

2851

In [13]:
max(c.values())

2

## Part 2
OK, that's going to take quite a while. However there's only a limited number of pairs of characters, so it should be possible to speed things up by caching the results. Could try `functools.lru_cache` and recursion but instead create a dictionary of counts, starting with a count of each pair with one step remaining, then from that deduce the count for each pair of characters with two steps remaining, and so on. 

In [34]:
from collections import defaultdict
from functools import reduce
from operator import add

def count_elements(steps, template, insertions):
    reverse_insertions = defaultdict(list)
    for k, v in insertions.items():
        reverse_insertions[v].append(k)
    counts_to = {}
    for x, y in insertions:
        insert = insertions[(x, y)]
        counts_to[(x, y, 1)] = Counter(insert)
    for i in range(2, steps + 1):
        for insert in reverse_insertions:
            for x, y in reverse_insertions[insert]:
                counts_to[(x, y, i)] = (
                    Counter(insert) + counts_to[(x, insert, i - 1)] 
                    + counts_to[(insert, y, i - 1)]
                )
    return reduce(add, (counts_to[(x, y, steps)] for x, y in zip(template, template[1:])), Counter()) + Counter(template)

def part_2(template, insertions):
    counts = count_elements(40, template, insertions)
    return max(counts.values()) - min(counts.values())

assert part_1(*test_data) == 1588

Counter({'C': 298, 'B': 1749, 'N': 865, 'H': 161})

In [35]:
part_2(*data)

10002813279337